In [108]:
import pandas as pd

df = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Pandas\superstore.csv', encoding = 'cp1252')
df.head(5)
df.info
df.shape
df.describe
df.isnull().sum()
df.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [109]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] =  pd.to_datetime(df['Ship Date'])

df['Ship Date'].dt.day_name().head(5)


0      Friday
1      Friday
2    Thursday
3      Sunday
4      Sunday
Name: Ship Date, dtype: object

**Handle Missing/Invalid Data**

In [110]:
df.duplicated().sum()
df.drop_duplicates()

df.isnull().sum()

essential_cols = ['Ship Date', 'Region']

df.dropna(subset=essential_cols).head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


**New Feature Engineering**

In [111]:
df['Delivery_Time_Days'] = df['Ship Date'] - df['Order Date']
df['Profit_Margin'] = df['Profit'] / df['Sales']
df['High_Profit_Flag'] = df['Profit_Margin'].apply(lambda x: 1 if x>0.2 else 0)
df['Shipping_Delay_Flag']= df['Delivery_Time_Days'].apply(lambda x: 1 if x> pd.Timedelta(days =5) else 0)

**Group-Level Aggregations**

Total_Sales = df.groupby(['Region','Country','Segment','Category']).agg({'Sales': 'sum','Profit': 'sum','Delivery_Time_Days': 'mean',                                                                      'Shipping_Delay_Flag': 'sum'})

Total_Sales

**Top Products Analysis**
➤ Top 10 products by total sales
➤ Top 10 products by profit margin

In [112]:
top_prod_sales = df.groupby('Product Name')['Sales'].sum().sort_values(ascending = False).head(10)
top_prod_profit = df.groupby('Product Name')['Profit_Margin'].sum().sort_values(ascending = False).head(10)
#df['month_year'] = pd.to_datetime (df['Order Date'], format = 'yyyy-mm')

df['Order_Month'] = df['Order Date'].dt.to_period('M')

x = df.groupby('Order_Month')[['Sales','Profit']].sum()

x = x.rename(columns ={'Sales': 'T_sales', 'Profit': 'T_Profit'})
df = df.merge(x, left_on = 'Order_Month', right_on ='Order_Month', how = 'left')
df.head(100)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Quantity,Discount,Profit,Delivery_Time_Days,Profit_Margin,High_Profit_Flag,Shipping_Delay_Flag,Order_Month,T_sales,T_Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,2,0.00,41.9136,3 days,0.160000,0,0,2016-11,79411.9658,4011.4075
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,3,0.00,219.5820,3 days,0.300000,1,0,2016-11,79411.9658,4011.4075
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,2,0.00,6.8714,4 days,0.470000,1,0,2016-06,40344.5340,4750.3781
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,5,0.45,-383.0310,7 days,-0.400000,0,1,2015-10,31404.9235,2817.3660
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2,0.20,2.5164,7 days,0.112500,0,1,2015-10,31404.9235,2817.3660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,US-2017-109484,2017-11-06,2017-11-12,Standard Class,RB-19705,Roger Barcio,Home Office,United States,Portland,...,1,0.70,-3.7880,6 days,-0.666667,0,1,2017-11,118447.8250,9690.1037
96,97,CA-2017-161018,2017-11-09,2017-11-11,Second Class,PN-18775,Parhena Norris,Home Office,United States,New York City,...,7,0.00,40.5426,2 days,0.420000,1,0,2017-11,118447.8250,9690.1037
97,98,CA-2017-157833,2017-06-17,2017-06-20,First Class,KD-16345,Katherine Ducich,Consumer,United States,San Francisco,...,3,0.20,17.9592,3 days,0.350000,1,0,2017-06,52981.7257,8223.3357
98,99,CA-2016-149223,2016-09-06,2016-09-11,Standard Class,ER-13855,Elpida Rittenbach,Corporate,United States,Saint Paul,...,6,0.00,22.5852,5 days,0.290000,1,0,2016-09,73410.0249,9328.6576


**Pivot Tables**

In [113]:
df.pivot_table(index = 'Region', columns ='Category', values = ['T_sales','T_Profit'], aggfunc = lambda x: f"{x.sum():,.2f}")

df.pivot_table(index = 'Region', columns = 'Ship Mode', values = 'Shipping_Delay_Flag', aggfunc = 'sum')

Ship Mode,First Class,Same Day,Second Class,Standard Class
Region,,,,
Central,0,0,0,443
East,0,0,0,496
South,0,0,0,284
West,0,0,0,601


**Country-Level Ranking**

In [133]:
df[['Region', 'T_sales']].sort_values(by='T_sales',ascending = False).groupby('Region').head(2)

df[['Region','Product Name','Delivery_Time_Days']].sort_values(by=['Delivery_Time_Days','Region'], ascending =[False,False])\
    .groupby('Region').head(10)

,Region,Product Name,Delivery_Time_Days
126,West,"Avery Durable Slant Ring Binders, No Labels",7 days
127,West,"Trav-L-File Heavy-Duty Shuttle II, Black",7 days
459,West,3M Office Air Cleaner,7 days
460,West,Logitech G700s Rechargeable Gaming Mouse,7 days
461,West,Acco D-Ring Binder w/DublLock,7 days
462,West,Eldon Wave Desk Accessories,7 days
463,West,Bush Advantage Collection Racetrack Conference...,7 days
464,West,Poly Designer Cover & Back,7 days
465,West,Premier Electric Letter Opener,7 days
466,West,"Fellowes Premier Superior Surge Suppressor, 10...",7 days
